# Question answering on the SQuAD dataset

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import dataset

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.xmargin'] = .05
plt.rcParams['axes.ymargin'] = .05
plt.style.use('ggplot')

## Data loading/preparation

In [49]:
def preprocess_question(text):
    """
    Preprocess the given question text from the SQuAD dataset
    """
    return text.lower()


def preprocess_context(text):
    """
    Preprocess the given context text from the SQuAD dataset
    """
    return text.lower()

In [50]:
squad_dataset = dataset.SquadDataset(
    max_question_tokens=25,
    max_context_tokens=250,
    question_preprocessor=preprocess_question,
    context_preprocessor=preprocess_context,
)
squad_dataset.dataframe

,index,question,title,context_id,context,text,answer_start,answer_end
0,5733be284776f41900661182,to whom did the virgin mary allegedly appear i...,University_of_Notre_Dame,0,"architecturally, the school has a catholic cha...",saint bernadette soubirous,515,541
1,5733be284776f4190066117f,what is in front of the notre dame main building?,University_of_Notre_Dame,0,"architecturally, the school has a catholic cha...",a copper statue of christ,188,213
2,5733be284776f41900661180,the basilica of the sacred heart at notre dame...,University_of_Notre_Dame,0,"architecturally, the school has a catholic cha...",the main building,279,296
3,5733be284776f41900661181,what is the grotto at notre dame?,University_of_Notre_Dame,0,"architecturally, the school has a catholic cha...",a marian place of prayer and reflection,381,420
4,5733be284776f4190066117e,what sits on top of the main building at notre...,University_of_Notre_Dame,0,"architecturally, the school has a catholic cha...",a golden statue of the virgin mary,92,126
...,...,...,...,...,...,...,...,...
87594,5735d259012e2f140011a09d,in what us state did kathmandu first establish...,Kathmandu,18890,"kathmandu metropolitan city (kmc), in order to...",oregon,229,235
87595,5735d259012e2f140011a09e,what was yangon previously known as?,Kathmandu,18890,"kathmandu metropolitan city (kmc), in order to...",rangoon,414,421
87596,5735d259012e2f140011a09f,with what belorussian city does kathmandu have...,Kathmandu,18890,"kathmandu metropolitan city (kmc), in order to...",minsk,476,481
87597,5735d259012e2f140011a0a0,in what year did kathmandu create its initial ...,Kathmandu,18890,"kathmandu metropolitan city (kmc), in order to...",1975,199,203


In [51]:
squad_dataset.train_df

,index,question,title,context_id,context,text,answer_start,answer_end
0,5733be284776f41900661182,to whom did the virgin mary allegedly appear i...,University_of_Notre_Dame,0,"architecturally, the school has a catholic cha...",saint bernadette soubirous,515,541
1,5733be284776f4190066117f,what is in front of the notre dame main building?,University_of_Notre_Dame,0,"architecturally, the school has a catholic cha...",a copper statue of christ,188,213
2,5733be284776f41900661180,the basilica of the sacred heart at notre dame...,University_of_Notre_Dame,0,"architecturally, the school has a catholic cha...",the main building,279,296
3,5733be284776f41900661181,what is the grotto at notre dame?,University_of_Notre_Dame,0,"architecturally, the school has a catholic cha...",a marian place of prayer and reflection,381,420
4,5733be284776f4190066117e,what sits on top of the main building at notre...,University_of_Notre_Dame,0,"architecturally, the school has a catholic cha...",a golden statue of the virgin mary,92,126
...,...,...,...,...,...,...,...,...
70185,5735d259012e2f140011a09d,in what us state did kathmandu first establish...,Kathmandu,18890,"kathmandu metropolitan city (kmc), in order to...",oregon,229,235
70186,5735d259012e2f140011a09e,what was yangon previously known as?,Kathmandu,18890,"kathmandu metropolitan city (kmc), in order to...",rangoon,414,421
70187,5735d259012e2f140011a09f,with what belorussian city does kathmandu have...,Kathmandu,18890,"kathmandu metropolitan city (kmc), in order to...",minsk,476,481
70188,5735d259012e2f140011a0a0,in what year did kathmandu create its initial ...,Kathmandu,18890,"kathmandu metropolitan city (kmc), in order to...",1975,199,203


In [52]:
squad_dataset.val_df

,index,question,title,context_id,context,text,answer_start,answer_end
0,56be85543aeaaa14008c9063,when did beyonce start becoming popular?,Beyoncé,55,beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ b...,in the late 1990s,269,286
1,56be85543aeaaa14008c9065,what areas did beyonce compete in when she was...,Beyoncé,55,beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ b...,singing and dancing,207,226
2,56be85543aeaaa14008c9066,when did beyonce leave destiny's child and bec...,Beyoncé,55,beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ b...,2003,526,530
3,56bf6b0f3aeaaa14008c9601,in what city and state did beyonce grow up?,Beyoncé,55,beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ b...,"houston, texas",166,180
4,56bf6b0f3aeaaa14008c9602,in which decade did beyonce become famous?,Beyoncé,55,beyoncé giselle knowles-carter (/biːˈjɒnseɪ/ b...,late 1990s,276,286
...,...,...,...,...,...,...,...,...
17404,5732868bb3a91d1900202e0f,at what augusta hole was the eisenhower pine l...,Dwight_D._Eisenhower,18592,"a loblolly pine, known as the ""eisenhower pine...",17th,74,78
17405,5732868bb3a91d1900202e10,how many meters away from the masters tee on a...,Dwight_D._Eisenhower,18592,"a loblolly pine, known as the ""eisenhower pine...",192,110,113
17406,5732868bb3a91d1900202e11,what did eisenhower want to be done to the eis...,Dwight_D._Eisenhower,18592,"a loblolly pine, known as the ""eisenhower pine...",cut down,279,287
17407,5732868bb3a91d1900202e12,what damaged the eisenhower pine in february 2...,Dwight_D._Eisenhower,18592,"a loblolly pine, known as the ""eisenhower pine...",ice storm,478,487


In [54]:
squad_dataset.train_dataset[0]

('to whom did the virgin mary allegedly appear in 1858 in lourdes france?',
 'architecturally, the school has a catholic character. atop the main building\'s gold dome is a golden statue of the virgin mary. immediately in front of the main building and facing it, is a copper statue of christ with arms upraised with the legend "venite ad me omnes". next to the main building is the basilica of the sacred heart. immediately behind the basilica is the grotto, a marian place of prayer and reflection. it is a replica of the grotto at lourdes, france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858. at the end of the main drive (and in a direct line that connects through 3 statues and the gold dome), is a simple, modern stone statue of mary.',
 515,
 541)

In [53]:
next(iter(squad_dataset.train_dataloader))

TypeError: new(): invalid data type 'str'